In [1]:
import os
import time
import json
import requests
from openpyxl import load_workbook
import pandas as pd

In [2]:
# http request authentication
with open('../../gitToken.txt', 'r') as f:
    token = "token " + f.readline()
f.close()
header = {'Authorization': token}

base_url = 'https://api.github.com/search/code?q=extension:xlsx'
page_url = '&per_page=100'

def search_xlsx_on_github(min_file_size, max_file_size, download_path, debug=False):
    search_url = base_url + '+size:' + str(min_file_size) + '..' + str(max_file_size) + page_url
    print(search_url)
    
    # query GitHub
    req = get_github_request(search_url, header)
    json = req.json()
    head = req.headers
    if debug:
        print('response: \n', req)
        print('headers: \n', head)
        print('JSON: \n', json)
        #print('response content: \n', r.content)
        print()
        print()
    
    total_seen, total_downloaded = download_xlsx(download_path, json, debug)
    
    while 'next' in req.links:
        next_url = req.links['next']['url']
        req = get_github_request(next_url, header)
        json = req.json()
        head = req.headers
        
        a, b = download_xlsx(download_path, json, debug)
        total_seen += a
        total_downloaded += b
    print('total seen:', total_seen + '\ntotal downloaded:', total_downloaded)
        
def get_github_request(url, header):
    r = requests.get(url, headers=header)
    if r.status_code == 403:
        wait = 0
        try:
            wait = r.headers['Retry-After']
        except KeyError:
            wait = 30
        print("%s: Hit rate limit. Retry after %s seconds" % (r.headers['Date'], wait))
        time.sleep(int(wait) + 1)
        return get_github_request(url, header)
    return r

def download_xlsx(download_path, json, debug):
    if debug:
        print(json)
    total_seen = 0
    total_downloaded = 0
    for item in json['items']:
        file_url = item['html_url']
        file_url = item['html_url']
        filename = item['name']
        req = get_github_request(file_url + '?raw=true', header)
        if not os.path.exists(download_path):
            os.makedirs(download_path)
        total_seen += 1
        with open(download_path + filename, 'wb')as f:
            f.write(req.content)
        f.close()
        if check_xlsx_for_formulae(download_path + filename):
            print('downloaded ' + filename)
            total_downloaded += 1
        else:
            print('ignored ' + filename)
            os.remove(download_path + filename)
    return total_seen, total_downloaded


cutoff = 0.3
def check_xlsx_for_formulae(file_path):
    try:
        wb = load_workbook(filename = file_path)
        sheet_names = wb.sheetnames
        num_cells = 1
        num_formulas = 0
        for sheet in sheet_names:
            ws = wb[sheet]
            print(wb)
            for row in ws.iter_rows():
                for cell in row:
                    if type(cell.value) is not None:
                        num_cells += 1
                    if len(str(cell.value)) > 0 and str(cell.value)[0] == '=':
                        num_formulas += 1
                    if num_formulas >= 10000:
                        wb.close()
                        return True
        wb.close()
        return num_formulas / num_cells > cutoff
    except:
        return False
    


In [ ]:
download_path = "../../xlsxDownloads/100000kb+/"
search_xlsx_on_github(100000000, 100000000000, download_path, debug=False)

https://api.github.com/search/code?q=extension:xlsx+size:100000000..100000000000&per_page=100
ignored AwsArquitectNotesCloudGuru.xlsx
ignored Comments.xlsx
ignored DATA_RELEVANT.xlsx
downloaded Technosylva-sizes-2021WMP_ClassB_Action-PGE-15_Atch01-mbar.xlsx
downloaded data_test.xlsx
ignored Master Policy - Viva Air.xlsx


C:\Users\jonat\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


ignored blackjackFinalData.xlsx
ignored ESI_2020.xlsx


C:\Users\jonat\anaconda3\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [ ]:
download_path = "../../xlsxDownloads/10000-100000kb/"
search_xlsx_on_github(10000000, 99999999, download_path, debug=False)

In [ ]:
download_path = "../../xlsxDownloads/1000-10000kb/"
search_xlsx_on_github(1000000, 9999999, download_path, debug=True)

In [ ]:
download_path = "../../xlsxDownloads/500-1000kb/"
search_xlsx_on_github(500000, 999999, download_path, debug=False)